# Football Logos Classification by League

**Objective:** Classify football team logos into their respective European leagues.

**Dataset:** 605 logos from 26 European leagues.

**Models:** Custom CNN baseline and ResNet18 transfer learning.

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import torch
import matplotlib.pyplot as plt

from src.dataset import get_dataloaders
from src.models import CustomCNN, get_resnet18
from src.train import train_model
from src.evaluate import evaluate_model, plot_confusion_matrix, plot_training_history
from src.utils import predict_from_dataset, visualize_prediction_from_dataset, visualize_dataset_samples

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

## 2. Load Dataset

In [ ]:
DATA_DIR = '../data'
BATCH_SIZE = 32

train_loader, val_loader, test_loader, class_names = get_dataloaders(
    DATA_DIR,
    batch_size=BATCH_SIZE,
    val_split=0.15,
    test_split=0.15
)

print(f"Number of classes (leagues): {len(class_names)}")
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")

## 3. Explore Dataset

In [ ]:
print("Leagues in dataset:")
for i, league in enumerate(class_names, 1):
    print(f"{i:2d}. {league}")

In [ ]:
visualize_dataset_samples(train_loader.dataset.dataset, class_names, n_samples=16)

## 4. Model 1: Custom CNN (Baseline)

In [ ]:
model1 = CustomCNN(num_classes=len(class_names))
print("Model 1: Custom CNN")
print(f"Total parameters: {sum(p.numel() for p in model1.parameters()):,}")

In [ ]:
history1 = train_model(
    model1,
    train_loader,
    val_loader,
    epochs=20,
    lr=0.001,
    device=DEVICE
)

In [ ]:
plot_training_history(history1)

## 5. Model 2: ResNet18 (Transfer Learning)

In [ ]:
model2 = get_resnet18(num_classes=len(class_names), pretrained=True)
print("Model 2: ResNet18 with ImageNet pretrained weights")
print(f"Total parameters: {sum(p.numel() for p in model2.parameters()):,}")

In [ ]:
history2 = train_model(
    model2,
    train_loader,
    val_loader,
    epochs=15,
    lr=0.0001,
    device=DEVICE
)

In [ ]:
plot_training_history(history2)

## 6. Evaluation and Comparison

In [ ]:
print("=" * 50)
print("MODEL 1: Custom CNN")
print("=" * 50)
results1 = evaluate_model(model1, test_loader, class_names, device=DEVICE)

In [ ]:
print("\n" + "=" * 50)
print("MODEL 2: ResNet18")
print("=" * 50)
results2 = evaluate_model(model2, test_loader, class_names, device=DEVICE)

In [ ]:
print("\n" + "=" * 50)
print("COMPARISON")
print("=" * 50)
print(f"Custom CNN Accuracy: {results1['accuracy']:.2f}%")
print(f"ResNet18 Accuracy: {results2['accuracy']:.2f}%")
print(f"Improvement: {results2['accuracy'] - results1['accuracy']:.2f}%")

## 7. Confusion Matrix (Best Model)

In [ ]:
best_results = results2 if results2['accuracy'] > results1['accuracy'] else results1
plot_confusion_matrix(
    best_results['labels'],
    best_results['predictions'],
    class_names,
    figsize=(14, 12)
)

## 8. Test Predictions on Random Samples

In [ ]:
import random

best_model = model2 if results2['accuracy'] > results1['accuracy'] else model1
test_dataset = test_loader.dataset.dataset

random_idx = random.randint(0, len(test_dataset) - 1)

predictions, true_label, image = predict_from_dataset(
    test_dataset,
    best_model,
    class_names,
    random_idx,
    device=DEVICE,
    top_k=5
)

print(f"True Label: {true_label}")
print("\nTop 5 predictions:")
for i, (league, prob) in enumerate(predictions, 1):
    print(f"{i}. {league}: {prob:.2f}%")

In [ ]:
visualize_prediction_from_dataset(image, predictions[:3], true_label)

## 9. Save Best Model

In [ ]:
MODEL_PATH = '../models/best_model.pth'
torch.save(best_model.state_dict(), MODEL_PATH)
print(f"Best model saved to {MODEL_PATH}")
print(f"Final test accuracy: {max(results1['accuracy'], results2['accuracy']):.2f}%")